# Data Cleaning

In [1]:
import pandas as pd

In [2]:
#Importing datasets

data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

#Display 10 columns from train.csv dataset to check what we're working with
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
#We will drop some of the rows to simplify things, this may impact the score, but we're keeping it simple.
#Ticket row doesn't serve much purpose in terms of predicting survivability, hence why we'll drop it.
#Cabin columns contain some missing values, some cabins may have higher survivability rate than others, but it's hard to predict.
#Name row does not serve much purpose in predicting survivability. Some passengers may be doctors, thus increasing their survivability, but we'll generalise that most passengers aren't doctors.
#PassengerId serves no purpose determining survivability.
def clean(data):
    data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis=1)

#As there are missing values, we will fill the values with mean numbers to not skew the data.    
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)

#Filling missing data for Embarked row with "U" which indicates "Unknown"        
    data.Embarked.fillna("U", inplace=True)
    return data

data = clean(data)
test = clean(test)

In [4]:
data.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,28.0,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


In [5]:
#Converting "Sex" and "Embarked" strings to numerical values.
#Female = 0, Male = 1
#C = 0, Q = 1, S = 2, U = 3
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]

for col in cols:
    data[col] = le.fit_transform(data[col])
    test[col] = le.transform(test[col])
    print(le.classes_)
    
data.head(10)

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2
5,0,3,1,28.0,0,0,8.4583,1
6,0,1,1,54.0,0,0,51.8625,2
7,0,3,1,2.0,3,1,21.0750,2
8,1,3,0,27.0,0,2,11.1333,2
9,1,2,0,14.0,1,0,30.0708,0


# Learning Model

For the Titanic dataset we will be using Logistical Regression for simplicity. Other models will work as well, better or worse.

In [8]:
#This is very important step, you need to hold part of your data (test set) to use after finishing the
#training process, to evaluate the performance of your model. This is one of the important way to
#ensure that your model is not overfitting. Below, we use the train_test_split to do this split (80%
#of the data for training, and 20% of the data for testing)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

y = data["Survived"]
X = data.drop("Survived", axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [9]:
#Training the model using the data
clf = LogisticRegression(random_state = 0, max_iter = 900).fit(X_train, y_train)

In [10]:
#Predicting on the validation set
predictions = clf.predict(X_val)

from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

0.8100558659217877

From the validation set we can see that there's a survivablity rate of 81% using logistical regression.

Logistical Regression is a simple model that we've used for this dataset. Perhaps a greater accuracy can be achieved by using models such as KNN, Decision Trees or Random Forests.